<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jayralencar/pictogram-to-sense/blob/main/Evaluation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jayralencar/pictogram-to-sense/blob/main/Evaluation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://colab.research.google.com/drive/1K8y-LbAafggaF5JR7q8Uo4qMDSs3YTF0"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# CLIP and fine-tuned CLIP evaluation

This notebook presents the code used for evaluating CLIP and the fine-tuned CLIP performance on mapping a pictogram to a WordNet synset.

## Requirements

In [ ]:
!git clone -b finetuning https://github.com/jayralencar/train-CLIP

Cloning into 'train-CLIP'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 236 (delta 137), reused 152 (delta 65), pack-reused 0
Receiving objects: 100% (236/236), 232.13 KiB | 14.51 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [ ]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
!pip install pytorch_lightning transformers

     |████████████████████████████████| 1.0 MB 28.6 MB/s 
     |████████████████████████████████| 3.1 MB 31.7 MB/s 
     |████████████████████████████████| 329 kB 49.7 MB/s 
     |████████████████████████████████| 596 kB 31.9 MB/s 
     |████████████████████████████████| 132 kB 57.7 MB/s 
     |████████████████████████████████| 829 kB 42.7 MB/s 
     |████████████████████████████████| 1.1 MB 26.7 MB/s 
     |████████████████████████████████| 3.3 MB 37.6 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 192 kB 56.4 MB/s 
     |████████████████████████████████| 271 kB 58.7 MB/s 
     |████████████████████████████████| 160 kB 47.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=55558edcc0c2670ce81803f33a880b815918df743b0f2578161bfba92a7efb91
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae3

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-26eggu7h
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-26eggu7h
     |████████████████████████████████| 64 kB 2.3 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369106 sha256=c4398e20f780d1d6ca7ad4ff96c16fac4840067abccb075058b1e8f8fca0e9d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-69v17xwo/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=b5a311ddf1c70e71cbb8f881f77324c85e79d292fc505b047aeb1cd2874c4072
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built clip ftfy


In [ ]:
pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'

  Cloning https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup to /tmp/pip-req-build-l8ql5a_x
  Running command git clone -q https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup /tmp/pip-req-build-l8ql5a_x
  Created wheel for cosine-annealing-warmup: filename=cosine_annealing_warmup-2.0-py3-none-any.whl size=3334 sha256=35b707a5113c36cfa2fe3b6ea51171004db022ef46968606983b1f7e49b3a4a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-r1d2hm3q/wheels/6c/b9/45/0fa58a1711c535236d946bbeff05d366eaf6818faed404625e
Successfully built cosine-annealing-warmup


In [ ]:
!mv train-CLIP/ train_clip

## Download testing dataset

We used the [Mulberry Symbols](https://mulberrysymbols.org/) as dataset for evaluation. The symbols consist of a set of 3,436 graphic images followed by labels and designed to communication use, especially for adults. We manually annotated 900 pictograms related to nouns, verbs, adjectives, and adverbs, using WordNet synsets. For this, we considered the labels provided to each pictogram and searched for their more appropriated synset in WordNet considering the action or entity present in the pictogram picture.  This way, our test dataset consist of a set of (pictogram, synset) pairs. As each synset has a glossary definition, we can expand the dataset to (pictogram, description) pairs.

In [ ]:
!gdown https://drive.google.com/uc?id=1AjxMS3IWGk2s0gahCU988o0Pdk2nR4fq
!unzip mulberry-symbols-PNG.zip

Downloading...
From: https://drive.google.com/uc?id=1AjxMS3IWGk2s0gahCU988o0Pdk2nR4fq
To: /content/mulberry-symbols-PNG.zip
100% 142M/142M [00:02<00:00, 65.2MB/s]
Archive:  mulberry-symbols-PNG.zip
   creating: PNG/
  inflating: PNG/playdough.png       
  inflating: PNG/eye_drops.png       
  inflating: PNG/communicate_,_to.png  
  inflating: PNG/sun.png             
  inflating: PNG/geography_class.png  
  inflating: PNG/mobile_phone_silent.png  
  inflating: PNG/sweet.png           
  inflating: PNG/sauce_cranberry.png  
  inflating: PNG/country_Samoa.png   
  inflating: PNG/cake_mix.png        
  inflating: PNG/picture.png         
  inflating: PNG/today.png           
  inflating: PNG/hare.png            
  inflating: PNG/magnet.png          
  inflating: PNG/milk_person_2a.png  
  inflating: PNG/country_Grenada.png  
  inflating: PNG/square_setsquare.png  
  inflating: PNG/swimming_costume.png  
  inflating: PNG/feed_cat_,_to.png   
  inflating: PNG/sit_2_,_to.png      
  inflatin

In [ ]:
!gdown https://drive.google.com/uc?id=1fk9Ws21n2ClVB_5Riz-s8syR0PUTw7_C

Downloading...
From: https://drive.google.com/uc?id=1fk9Ws21n2ClVB_5Riz-s8syR0PUTw7_C
To: /content/ambiguous.json
100% 1.47M/1.47M [00:00<00:00, 22.8MB/s]


## Prepare items

In [ ]:
import json
pictograms = json.load(open('./ambiguous.json'))
pictograms = [p for p in pictograms if 'synset' in p]
len(pictograms)

958

In [ ]:
noun_pictograms = [p for p in pictograms if wn.synset(p['synset']).pos() == 'n']
verb_pictograms = [p for p in pictograms if wn.synset(p['synset']).pos() == 'v']
adverb_pictograms = [p for p in pictograms if wn.synset(p['synset']).pos() == 'r']
adjective_pictograms = [p for p in pictograms if wn.synset(p['synset']).pos() in ['a','s']]
len(noun_pictograms), len(verb_pictograms), len(adverb_pictograms), len(adjective_pictograms)

(642, 214, 24, 78)

## Evaluating CLIP

In [ ]:
import clip
import torch
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
clip.available_models()

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'ViT-B/32', 'ViT-B/16']

In [ ]:
for model_name in clip.available_models():
  print("Model name",model_name)
  print("> Downloading and loading model")
  model, preprocess = clip.load(model_name, device=device)
  true_y = []
  predicted_y = []
  print('> inference')
  picts = [noun_pictograms, verb_pictograms, adverb_pictograms, adjective_pictograms]
  labels = ['Nouns','Verbs',"Adverbs",'Adjectives']
  
  for i, label in enumerate(labels):
    print(">>",label)
    true_l = []
    predicted_l = []
    pbar = tqdm(picts[i])
    for pictogram in pbar:
      file_path = "./PNG/{0}".format(pictogram['filename'])
      pil_image = Image.open(file_path)
      image = preprocess(pil_image).unsqueeze(0).to(device)
      pict_s = wn.synset(pictogram['synset'])
      # definitions = [s['definition'] for s in pictogram['synsets']]
      # synsets = [s['synset'] for s in pictogram['synsets']]
      synsets = wn.synsets(pictogram['word'], pict_s.pos())
      
      definitions = []
      for ss in synsets:
          # print(sss)
          # ss = wn.synset(sss)
          defs = [ss.definition()]
          for e in ss.examples():
              defs.append(e)
          # for l in ss.lemmas():
          #     defs.append(l.name())
          definitions.append(". ".join(defs))
      text = clip.tokenize(definitions, truncate=True).to(device)
      with torch.no_grad():
          logits_per_image, logits_per_text = model(image, text)
          probs = logits_per_image.softmax(dim=-1).cpu().numpy()
      better = probs.argmax()

      predicted_y.append(synsets[better].name())
      predicted_l.append(synsets[better].name())
      true_y.append(pictogram['synset'])
      true_l.append(pictogram['synset'])

      acc = accuracy_score(true_l,predicted_l)
      pbar.set_postfix({'acc': acc})
    print("Accuracy ", label,":",accuracy_score(true_l,predicted_l))
  print("Accuracy:",accuracy_score(true_y,predicted_y))

Model name RN50
> Downloading and loading model
> inference
>> Nouns


100%|██████████| 642/642 [07:11<00:00,  1.49it/s, acc=0.626]


Accuracy  Nouns : 0.6261682242990654
>> Verbs


100%|██████████| 214/214 [04:14<00:00,  1.19s/it, acc=0.374]


Accuracy  Verbs : 0.37383177570093457
>> Adverbs


...

Accuracy  Verbs : 0.40186915887850466
>> Adverbs


100%|██████████| 24/24 [00:23<00:00,  1.01it/s, acc=0.208]


Accuracy  Adverbs : 0.20833333333333334
>> Adjectives


100%|██████████| 78/78 [01:40<00:00,  1.29s/it, acc=0.462]

Accuracy  Adjectives : 0.46153846153846156
Accuracy: 0.6012526096033403


## Evaluating Fine-tuned

In [ ]:
!gdown https://drive.google.com/uc?id=1iNAamnjAmiggV8nXjGr40AxzjApWQZ9E
!unzip all.zip

Downloading...
From: https://drive.google.com/uc?id=1iNAamnjAmiggV8nXjGr40AxzjApWQZ9E
To: /content/all.zip
100% 6.31G/6.31G [02:03<00:00, 51.1MB/s]
Archive:  all.zip
   creating: all/
  inflating: all/ViT-B 32.ckpt       
  inflating: all/RN50.ckpt           
  inflating: all/RN101.ckpt          
  inflating: all/RN50x4.ckpt         
  inflating: all/RN50x16.ckpt        
  inflating: all/ViT-B 16.ckpt       


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
import math
import yaml
import copy
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
import clip

class CLIPFinetuningWrapper(pl.LightningModule):
    # Module based on
    def __init__(self, model_name,batch_size=1, learning_rate=3e-3,):
        super().__init__()
        #hparams
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        # Load CLIP
        # print("MODEL INSIDE", model_name)
        self.model, self.preprocess = clip.load(model_name, device=self.device)
        
        # Prepare for loss calculating
        self.loss_img = nn.CrossEntropyLoss()
        self.loss_txt = nn.CrossEntropyLoss()

    def forward(self, images, texts):
        return self.model(images, texts)

    def training_step(self, train_batch, idx):
        image, text = train_batch
        
        n = math.ceil(len(image) // self.batch_size)
        image_mbs = torch.chunk(image, n)

        ims = [F.normalize(self.model.encode_image(im), dim=1) for im in image_mbs]
        # gather from all GPUs
        ims = self.all_gather(torch.cat(ims))

        logits_per_image, logits_per_text = self.forward(image,text)
        ground_truth = torch.arange(self.batch_size,dtype=torch.long,device=self.device)

        loss = (self.loss_img(logits_per_image,ground_truth) + self.loss_txt(logits_per_text,ground_truth))/2

        self.log("train_loss", loss, on_epoch=True, prog_bar=True,)

        acc_i = (torch.argmax(logits_per_image, 0) == ground_truth).sum()
        acc_t = (torch.argmax(logits_per_image.t(), 0) == ground_truth).sum()
        acc = (acc_i + acc_t) / 2 / len(image)
        self.log("train_acc", acc, on_epoch=True, prog_bar=True,)
        # acc_i tensor(79, device='cuda:0')
        # acc_t tensor(62, device='cuda:0')
        # acc tensor(0.0043, device='cuda:0')
        return loss


    def validation_step(self, batch, idx):
        image, text = batch

        # print(self.batch_size)
        # /" ".join(model_name.split("/"))
        if MODE=='tune':
          n = len(image)
        else:
          n = math.ceil(len(image) // self.batch_size)
        # 
        # print(n)
        image_mbs = torch.chunk(image, n)
        
        ims = [F.normalize(self.model.encode_image(im), dim=1) for im in image_mbs]
        # gather from all GPUs
        ims = self.all_gather(torch.cat(ims))

        image_logits, text_logits = self.forward(image, text)
        ground_truth = torch.arange(len(image_logits)).to('cuda:0')
        loss = (F.cross_entropy(image_logits, ground_truth) + F.cross_entropy(text_logits, ground_truth)).div(2)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)

        acc_i = (torch.argmax(image_logits, 0) == ground_truth).sum()
        acc_t = (torch.argmax(image_logits.t(), 0) == ground_truth).sum()
        acc = (acc_i + acc_t) / 2 / len(image)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True,)

    def test_step(self, batch, idx):
        pass

    def configure_optimizers(self):
        lr = self.learning_rate

        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=lr,
            momentum=0.9
        )

        # print(self.num_training_steps)
        # print(int(self.num_training_steps*0.2))
        # Source: https://github.com/openai/CLIP/issues/107
        # Use pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'
        lr_scheduler = CosineAnnealingWarmupRestarts(
            optimizer,
            first_cycle_steps=50,
            cycle_mult=1.0,
            max_lr=lr,
            min_lr=0,
            warmup_steps=int(50*0.1)
        )

        return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
    
    # Sourced from https://github.com/PyTorchLightning/pytorch-lightning/issues/5449
    @property
    def num_training_steps(self) -> int:
        """Total training steps inferred from datamodule and devices."""
        dataset = self.train_dataloader()
        if self.trainer.max_steps:
            return self.trainer.max_steps

        dataset_size = len(dataset.dataset)

        # print("DS:", dataset_size)

        num_devices = max(1, self.trainer.num_gpus, self.trainer.num_processes)
        if self.trainer.tpu_cores:
            num_devices = max(num_devices, self.trainer.tpu_cores)

        effective_batch_size = dataset.batch_size * self.trainer.accumulate_grad_batches * num_devices
        # print("effective_batch_size",effective_batch_size)
        # a = (dataset_size // effective_batch_size) * self.trainer.max_epochs
        a = dataset_size // effective_batch_size
        # print('a',a)
        return a

In [ ]:
from torchvision import transforms as T
import clip

def fix_img(img):
    return img.convert('RGB') if img.mode != 'RGB' else img

image_transform = T.Compose([
    T.Lambda(fix_img),
    T.RandomResizedCrop(224,
                        scale=(0.75, 1.),
                        ratio=(1., 1.)),
    T.ToTensor(),
    T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

In [ ]:
import torch
from PIL import Image
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [ ]:
clip.available_models()

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'ViT-B/32', 'ViT-B/16']

In [ ]:
for model_name in clip.available_models():
# for model_name in ["ViT-B/32"]:
  print("model", model_name)
  # model_path = "./final.ckpt"
  model_path = "./all/{0}.ckpt".format(" ".join(model_name.split("/")))

  
  model = CLIPFinetuningWrapper.load_from_checkpoint(model_path, model_name="ViT-B/32")
  true_y = []
  predicted_y = []
  picts = [noun_pictograms, verb_pictograms, adverb_pictograms, adjective_pictograms]
  labels = ['Nouns','Verbs',"Adverbs",'Adjectives']
  
  for i, label in enumerate(labels):
    print(">>",label)
    true_l = []
    predicted_l = []
    pbar = tqdm(picts[i])
    for pictogram in pbar:
      # pictogram = pictograms[i]
      file_path = "./PNG/{0}".format(pictogram['filename'])
      with torch.no_grad():
        img_emb = image_transform(Image.open(file_path)).unsqueeze(0).to('cpu')
        # img_enc = model.model.encode_image(img_emb)
        # print(img_enc.size())
        # texts = [s['definition'] for s in pictogram['synsets']]
        pict_s = wn.synset(pictogram['synset'])
        # synsets = [s['synset'] for s in pictogram['synsets']]
        synsets = wn.synsets(pictogram['word'], pict_s.pos())
        texts = []
        for ss in synsets:
          # ss = wn.synset(s)
          # name,_,_ = ss.name().split(".")
          defs = [ss.definition()]
          # defs =[ss.definition()]
          # for h in ss.hypernyms():
          #   defs.append(h.definition())
          # for e in ss.examples():
          #     defs.append(e)
          # for l in ss.lemmas():
          #     defs.append(l.name())
          texts.append(", ".join(defs))
        
        text_emb= clip.tokenize(texts, truncate=True).to('cpu')
        # text_emb = tokenizer(texts,padding='max_length', truncation=True, return_tensors="pt",max_length=77)['input_ids'].to('cpu')
        # text_emb = tokenizer(texts,padding='max_length', truncation=True, return_tensors="pt",max_length=77)
        # print(text_emb)/
        # text_enc = model.model.encode_text(text_emb)
        # print(text_enc.size())
        # print(synsets,pictogram['word'],pictogram['synset'])
        logits_per_image, logits_per_text = model(img_emb,text_emb)
        # print(logits_per_image.size())
        # print(logits_per_text.size())
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        # probs = logits_per_image.cpu().numpy()
        better = probs.argmax()

        predicted_y.append(synsets[better].name())
        predicted_l.append(synsets[better].name())
        true_y.append(pictogram['synset'])
        true_l.append(pictogram['synset'])

        acc = accuracy_score(true_l,predicted_l)
        pbar.set_postfix({'acc': acc})
    # print("Accuracy ", label,":",accuracy_score(true_l,predicted_l))
  print("Accuracy:", accuracy_score(true_y,predicted_y))
  print("")
      # text = clip.tokenize(definitions, truncate=True).to(device)
  

model RN50
>> Nouns


100%|██████████| 642/642 [06:44<00:00,  1.59it/s, acc=0.769]


>> Verbs


100%|██████████| 214/214 [04:06<00:00,  1.15s/it, acc=0.453]


>> Adverbs


...

>> Adverbs


100%|██████████| 24/24 [00:14<00:00,  1.60it/s, acc=0.583]


>> Adjectives


100%|██████████| 78/78 [01:13<00:00,  1.06it/s, acc=0.577]

Accuracy: 0.6691022964509394



definition = 48%
name ; definition = 



In [ ]:
for i, true in enumerate(true_y):
  if true != predicted_y[i]:
    s_true = wn.synset(true)
    print(adjective_pictograms[i]['filename'])
    print("TRUE: ",true, s_true.definition())
    s_predicted = wn.synset(predicted_y[i])
    print("PRED: ",predicted_y[i], s_predicted.definition())
    print()

hot.png
TRUE:  hot.a.01 used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning
PRED:  hot.a.03 extended meanings; especially of psychological heat; marked by intensity or vehemence especially of passion or enthusiasm

whole.png
TRUE:  whole.a.01 including all components without exception; being one unit or constituting the full amount or extent or duration; complete
PRED:  solid.s.15 acting together as a single undiversified whole

long.png
TRUE:  long.a.02 primarily spatial sense; of relatively great or greater than average spatial extension or extension as specified
PRED:  long.a.01 primarily temporal sense; being or indicating a relatively great or greater than average duration or passage of time or a duration as specified

strong.png
TRUE:  strong.a.01 having strength or power greater than average or expected
PRED:  potent.s.02 having or wielding force or authority

full.png
TRUE:  full.a.01 